In [ ]:

# 📦 INSTALL (jika perlu di Colab)
# !pip install tensorflow pandas matplotlib scikit-learn

# 🧩 IMPORT LIBRARY
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, SimpleRNN, GRU, Dense

# 🧪 SIMULASI DATA TIME SERIES ROBOT
np.random.seed(42)
days = pd.date_range(start="2025-01-01", periods=100, freq='D')
activity_level = np.abs(np.sin(np.linspace(0, 3*np.pi, 100)) + np.random.normal(0, 0.3, 100)) * 10

df = pd.DataFrame({
    "date": days,
    "robot_id": "R1",
    "activity_level": activity_level
})

# 🔧 PREPROCESSING
df.set_index("date", inplace=True)
scaler = MinMaxScaler()
df["activity_scaled"] = scaler.fit_transform(df[["activity_level"]])

# 📈 MEMBUAT WINDOW TIME SERIES
def create_dataset(data, look_back=5):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:i+look_back])
        y.append(data[i+look_back])
    return np.array(X), np.array(y)

look_back = 5
X, y = create_dataset(df["activity_scaled"].values, look_back)
X = X.reshape((X.shape[0], X.shape[1], 1))

# 🧪 SPLIT TRAIN/TEST
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# 🧠 FUNGSI LATIH MODEL
def build_and_train(model_type, X_train, y_train, X_test, y_test, units=50, epochs=20):
    model = Sequential()
    if model_type == 'LSTM':
        model.add(LSTM(units, input_shape=(X_train.shape[1], 1)))
    elif model_type == 'RNN':
        model.add(SimpleRNN(unput_shape=(X_train.shape[1], 1), units=units))
    elif model_type == 'GRU':
        model.add(GRU(units, input_shape=(X_train.shape[1], 1)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=0)
    return model, history

# 🧠 LATIH MODEL
model_lstm, hist_lstm = build_and_train('LSTM', X_train, y_train, X_test, y_test)
model_rnn, hist_rnn = build_and_train('RNN', X_train, y_train, X_test, y_test)
model_gru, hist_gru = build_and_train('GRU', X_train, y_train, X_test, y_test)

# 📊 PLOT LOSS
def plot_loss(histories, labels):
    plt.figure(figsize=(10, 5))
    for hist, label in zip(histories, labels):
        plt.plot(hist.history['val_loss'], label=f'{label} Val Loss')
        plt.plot(hist.history['loss'], linestyle='--', label=f'{label} Train Loss')
    plt.title("Model Comparison: Loss over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("MSE Loss")
    plt.legend()
    plt.grid(True)
    plt.show()

plot_loss([hist_lstm, hist_rnn, hist_gru], ['LSTM', 'RNN', 'GRU'])
